# Assignment 3 Question 1

### CO 487/687 Applied Cryptography Fall 2023 

This Jupyter notebook contains Python 3 code for Assignment 3 Question 1 on "Symemtric Encryption in Python".

### Documentation

- [Python cryptography library](https://cryptography.io/en/latest/)

The following code imports all the required functions for the assignment.

In [1]:
import base64
import getpass
import json
import os
import sys
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes    
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives import constant_time

These two functions convert a byte array into a printable string and back, which might be helpful to you since cryptographic routines often work with byte arrays.

In [2]:
def bytes2string(b):
    return base64.urlsafe_b64encode(b).decode('utf-8')

def string2bytes(s):
    return base64.urlsafe_b64decode(s.encode('utf-8'))

Implement the main encryption function below. Your function will take as input a string, and will output a string or dictionary containing all the values needed to decrypt (other than the password, of course). The code below will prompt the user to enter their password during encryption.

In [3]:
def encrypt(message):
    
    # encode the string as a byte string, since cryptographic functions usually work on bytes
    plaintext = message.encode('utf-8')

    # Use getpass to prompt the user for a password
    password = getpass.getpass("Enter password:")
    password2 = getpass.getpass("Enter password again:")

    # Do a quick check to make sure that the password is the same!
    if password != password2:
        sys.stderr.write("Passwords did not match")
        sys.exit()

    ### START: This is what you have to change
    '''
    Encryption: 
    1. Prepare the key for HMAC (k_for_mac) using the password 
    2. Prepare the key for encryption (k_for_encryption) using the key for HMAC 
    3. Pad the plaintext to ensure that it can conform to block size of 16 bytes (128 bits) in AES
    4. Perform encryption using AES and k_for_encryption on the padded plaintext to get ciphertext
    5. Obtain the mac_tag using HMAC on the ciphertext and k_for_mac (this essentially performs EtM)
    6. Return the ciphertext, mac_tag, and other configurations that are not secret, but needed for decryption
    '''
    # STEP 1 
    salt = os.urandom(16)

    kdf_for_mac = PBKDF2HMAC(
        algorithm=hashes.SHA3_256(), 
        length=16, # we need 128-bits == 16 bytes for AES128
        salt=salt,
        iterations=200000 
    )

    pwd_encoded = password.encode('utf-8')
    k_for_mac= kdf_for_mac.derive(pwd_encoded) 

    kdf_for_encryption = PBKDF2HMAC(
        algorithm=hashes.SHA3_256(), 
        length=16, # we need 128-bits == 16 bytes for AES128
        salt=salt,
        iterations=200000 
    )

    # STEP 2
    k_for_encryption=kdf_for_encryption.derive(k_for_mac)

    nonce_for_cipher = os.urandom(16) # the nonce has to be the same size as the block size of AES == 128
    cipher = Cipher(algorithms.AES(k_for_encryption), modes.CTR(nonce_for_cipher))
    encryptor = cipher.encryptor()

    # STEP 3
    padder = padding.PKCS7(16).padder() 
    padded_data = padder.update(plaintext) + padder.finalize() 

    # STEP 4
    ct = encryptor.update(padded_data) + encryptor.finalize()
    
    # STEP 5
    h = hmac.HMAC(k_for_mac, hashes.SHA256())
    h.update(ct)
    mac_tag = h.finalize()

    # STEP 6 
    return {"ct": ct, "mac_tag": mac_tag, "salt": salt, "nonce_for_cipher": nonce_for_cipher}
    
    ### END: This is what you have to change

Now we call the `encrypt` function with a message, and print out the ciphertext it generates.

In [4]:
mymessage = "Hello, world!"
ciphertext = encrypt(mymessage)

Enter password:········
Enter password again:········


Implement the main decryption function below.  Your function will take as input the string or dictionary output by `encrypt`, prompt the user to enter the password, and then do all the relevant cryptographic operations.

In [5]:
def decrypt(ciphertext):
    
    # prompt the user for the password
    password = getpass.getpass("Enter the password:")

    ### START: This is what you have to change
    ''''''
    '''
    Decryption: 
    1. Extract important constants from the ciphertext dictionary
    2. Use the password inputted by user, to derive k_for_mac 
    3. Check whether the mac_tag is valid based on the k_for_mac and the ciphertext ct 
    4. If valid, then continue
    5. Derive k_for_decryption using k_for_mac
    6. Decrypt ct, using k_for_decryption
    7. Return the decrypted text 
    '''
    # STEP 1 
    # get the salt 
    salt = ciphertext["salt"]
    # get the mac_tag
    mac_tag = ciphertext["mac_tag"]
    # ciphertext containing message
    ct = ciphertext["ct"]

    # STEP 2
    kdf_for_mac = PBKDF2HMAC(
        algorithm=hashes.SHA3_256(), 
        length=16, # we need 128-bits == 16 bytes for AES128
        salt=salt,
        iterations=200000 
    )
    
    pwd_encoded = password.encode('utf-8')
    k_for_mac= kdf_for_mac.derive(pwd_encoded)

    # STEP 3
    h = hmac.HMAC(k_for_mac, hashes.SHA256())
    h.update(ct)
    h.verify(mac_tag)
    
    # STEP 4 note that, h.verify raises an exception if verification fails  

    # STEP 5
    # get key for decryption 
    kdf_for_decryption = PBKDF2HMAC(
        algorithm=hashes.SHA3_256(), 
        length=16, # we need 128-bits == 16 bytes for AES128
        salt=salt,
        iterations=200000 
    )

    k_for_decryption = kdf_for_decryption.derive(k_for_mac)

    # STEP 6 
    # get the nonce 
    nonce_for_cipher = ciphertext["nonce_for_cipher"]
    
    cipher = Cipher(algorithms.AES(k_for_decryption), modes.CTR(nonce_for_cipher))
    decryptor = cipher.decryptor()

    plaintext = decryptor.update(ct)
    ### END: This is what you have to change
    # decode the byte string back to a string
    # STEP 7
    return plaintext.decode('utf-8')

Now let's try decrypting the ciphertext you encrypted above by entering the same password as you used for encryption.

In [6]:
mymessagedecrypted = decrypt(ciphertext)
print(mymessagedecrypted)
assert mymessagedecrypted == mymessage

Enter the password:········
Hello, world!


AssertionError: 

Try again but this time see what happens if you use a different password to decrypt. Your function should fail.

In [ ]:
mymessagedecrypted = decrypt(ciphertext)
print(mymessagedecrypted)
assert mymessagedecrypted == mymessage